In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import sys, collections, os, argparse
%matplotlib inline  
%config InlineBackend.figure_format = 'retina'


# Download the 10x Dataset `6k 1:1 Mixture of Fresh Frozen Human (HEK293T) and Mouse (NIH3T3) Cells`

10x datasets are available at
https://support.10xgenomics.com/single-cell-gene-expression/datasets

The page for the `6k 1:1 Mixture of Fresh Frozen Human (HEK293T) and Mouse (NIH3T3) Cells` dataset is
https://support.10xgenomics.com/single-cell-gene-expression/datasets/2.1.0/hgmm_6k

The FASTQ files (38G) can be downloaded with `curl` directly from http://http://s3-us-west-2.amazonaws.com/10x.files/samples/cell-exp/2.1.0/hgmm_6k/hgmm_6k_fastqs.tar

In the cell below we check if the dataset file `hgmm_6k_fastqs.tar` already exists. If not we download the dataset to the same directory as this notebook


In [ ]:
#Check if the file was downloaded already before doing curl:
if not (os.path.isfile('./hgmm_6k_fastqs.tar')): 
    # the `!` means we're running a command line statement (rather than python)  
    !curl -O http://s3-us-west-2.amazonaws.com/10x.files/samples/cell-exp/2.1.0/hgmm_6k/hgmm_6k_fastqs.tar
else: print('Dataset already downloaded!')

## untar the fastq files on hgmm_6k_fastqs folder
Note that this dataset was sequenced from 8 10x lanes (L001-L008).
Hence it has 24 files: read 1 (R1), read 2 (R1), and index (I1) for each lane 

In [ ]:
!tar -xvf ./hgmm_6k_fastqs.tar

# Buiding the kallisto index

First make sure that kallisto is installed and the version is greater than 0.45

If it's not installed, see instructions at https://pachterlab.github.io/kallisto/download

In [ ]:
!kallisto version 

First we build the kallisto index for the dataset. 
For this index in particular, because this is a species mixing experiment, we have to download the human and mouse transcriptome, concatenate them, and then build the index.
Building the index takes a few minutes and needs to be done only once.

### Download human and mouse reference transcriptomes from ensembl
In order to do that we first download the human and mouse transcriptomes from ensembl. You can see the reference genomes they have at https://uswest.ensembl.org/info/data/ftp/index.html

In [ ]:
#Check if the file was downloaded already before doing curl:
if not (os.path.isfile('Mus_musculus.GRCm38.cdna.all.fa.gz')): 
    # the `!` means we're running a command line statement (rather than python)  
    !curl -O ftp://ftp.ensembl.org/pub/release-94/fasta/mus_musculus/cdna/Mus_musculus.GRCm38.cdna.all.fa.gz
else: print('Mouse transcriptome already downloaded!')

if not (os.path.isfile('Homo_sapiens.GRCh38.cdna.all.fa.gz')): 
    # the `!` means we're running a command line statement (rather than python)  
    !curl -O ftp://ftp.ensembl.org/pub/release-94/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz
else: print('Human transcriptome already downloaded!')


In [ ]:
#concatenate the human and mouse transcriptomes
!zcat Homo_sapiens.GRCh38.cdna.all.fa.gz Mus_musculus.GRCm38.cdna.all.fa.gz | gzip -1 - > human_mouse_contatenated_transcriptome.fa.gz
print('Human and mouse transcriptomes concatenated!')

In [ ]:
### Now we can build the index
if not (os.path.isfile('human_mouse_transcriptome_index.idx')): 
    !kallisto index -i human_mouse_transcriptome_index.idx human_mouse_contatenated_transcriptome.fa.gz
else: print ('Human-mouse transcript index already exist!')

# Preparing transcript_to_gene.tsv file process the single cell data with kallisto bus

Depending on which transcriptome you used, you will need to create a file translating transcripts to genes. This notebook assumes the file is in `transcript_to_gene.tsv`, for ensembl transcriptomes these can be generated using biomart.

The general format of `transcript_to_gene.tsv` is

```
ENST00000632684.1	ENSG00000282431.1
ENST00000434970.2	ENSG00000237235.2
ENST00000448914.1	ENSG00000228985.1
ENST00000415118.1	ENSG00000223997.1
ENST00000631435.1	ENSG00000282253.1
...
```

To create the `transcript_to_gene.tsv` we fetch and parse the mouse GTF file from ensembl.

The reference GTF files are available at https://uswest.ensembl.org/info/data/ftp/index.html

The mouse ones which we use are at ftp://ftp.ensembl.org/pub/release-94/gtf/mus_musculus

In [ ]:
#Check if the file was downloaded already before doing curl:
if not (os.path.isfile('Mus_musculus.GRCm38.94.gtf.gz')): 
    # the `!` means we're running a command line statement (rather than python)  
    !curl -O ftp://ftp.ensembl.org/pub/release-94/gtf/mus_musculus/Mus_musculus.GRCm38.94.gtf.gz
else: print('Mouse GTF already downloaded!')
    
    
#Check if the file was downloaded already before doing curl:
if not (os.path.isfile('Homo_sapiens.GRCh38.94.gtf.gz')): 
    # the `!` means we're running a command line statement (rather than python)  
    !curl -O ftp://ftp.ensembl.org/pub/release-94/gtf/homo_sapiens/Homo_sapiens.GRCh38.94.gtf.gz
else: print('Human GTC already downloaded!')


In [ ]:
# Unzip the files
!gunzip -v ./Mus_musculus.GRCm38.94.gtf.gz
!gunzip -v ./Homo_sapiens.GRCh38.94.gtf.gz

In [ ]:
# concatenate the GTF files
!cat ./Mus_musculus.GRCm38.94.gtf ./Homo_sapiens.GRCh38.94.gtf > ./human_mouse_contatenated_GTF.gtf
print('Human and mouse GTF files concatenated!')

## Create transcript_to_gene.tsv

Now we can use the cells below to parse the GTF file and keep only the transcript mapping as a tsv file in the format below.
```
ENST00000632684.1	ENSG00000282431.1
ENST00000434970.2	ENSG00000237235.2
ENST00000448914.1	ENSG00000228985.1
```

In [ ]:
def create_transcript_list(input, use_name = True, use_version = True):
    r = {}
    for line in input:
        if len(line) == 0 or line[0] == '#':
            continue
        l = line.strip().split('\t')
        if l[2] == 'transcript':
            info = l[8]
            d = {}
            for x in info.split('; '):
                x = x.strip()
                p = x.find(' ')
                if p == -1:
                    continue
                k = x[:p]
                p = x.find('"',p)
                p2 = x.find('"',p+1)
                v = x[p+1:p2]
                d[k] = v


            if 'transcript_id' not in d or 'gene_id' not in d:
                continue

            tid = d['transcript_id']
            gid = d['gene_id']
            if use_version:
                if 'transcript_version' not in d or 'gene_version' not in d:
                    continue

                tid += '.' + d['transcript_version']
                gid += '.' + d['gene_version']
            gname = None
            if use_name:
                if 'gene_name' not in d:
                    continue
                gname = d['gene_name']

            if tid in r:
                continue

            r[tid] = (gid, gname)
    return r



def print_output(output, r, use_name = True):
    for tid in r:
        if use_name:
            output.write("%s\t%s\t%s\n"%(tid, r[tid][0], r[tid][1]))
        else:
            output.write("%s\t%s\n"%(tid, r[tid][0]))

In [ ]:
with open('./human_mouse_contatenated_GTF.gtf') as file:
    r = create_transcript_list(file, use_name = True, use_version = True)
with open('human_mouse_transcript_to_gene.tsv', "w+") as output:
    print_output(output, r, use_name = True)
print('Created human_mouse_transcript_to_gene.tsv file')

In [ ]:
with open('./Mus_musculus.GRCm38.94.gtf') as file:
    r = create_transcript_list(file, use_name = True, use_version = True)
with open('mouse_transcript_to_gene.tsv', "w+") as output:
    print_output(output, r, use_name = True)
print('Created mouse_transcript_to_gene.tsv file')

In [ ]:
with open('./Homo_sapiens.GRCh38.94.gtf') as file:
    r = create_transcript_list(file, use_name = True, use_version = True)
with open('human_transcript_to_gene.tsv', "w+") as output:
    print_output(output, r, use_name = True)
print('Created human_transcript_to_gene.tsv file')

# Run kallisto bus
kallisto bus supports several single cell sequencing technologies, as you can see below. We'll be using 10xv2            

In [ ]:
!kallisto bus --list

We are using paired end reads (R1 and R2 files) with 8 samples

In [ ]:
!kallisto bus -i human_mouse_transcriptome_index.idx  -o out_hgmm_6k -x 10xv2 -t 4 \
./fastqs/hgmm_6k_S1_L001_R1_001.fastq.gz ./fastqs/hgmm_6k_S1_L001_R2_001.fastq.gz \
./fastqs/hgmm_6k_S1_L002_R1_001.fastq.gz ./fastqs/hgmm_6k_S1_L002_R2_001.fastq.gz \
./fastqs/hgmm_6k_S1_L003_R1_001.fastq.gz ./fastqs/hgmm_6k_S1_L003_R2_001.fastq.gz \
./fastqs/hgmm_6k_S1_L004_R1_001.fastq.gz ./fastqs/hgmm_6k_S1_L004_R2_001.fastq.gz \
./fastqs/hgmm_6k_S1_L005_R1_001.fastq.gz ./fastqs/hgmm_6k_S1_L005_R2_001.fastq.gz \
./fastqs/hgmm_6k_S1_L006_R1_001.fastq.gz ./fastqs/hgmm_6k_S1_L006_R2_001.fastq.gz \
./fastqs/hgmm_6k_S1_L007_R1_001.fastq.gz ./fastqs/hgmm_6k_S1_L007_R2_001.fastq.gz \
./fastqs/hgmm_6k_S1_L008_R1_001.fastq.gz ./fastqs/hgmm_6k_S1_L008_R2_001.fastq.gz 

### The `matrix.ec` file

The `matrix.ec` is generated by kallisto and connects the equivalence class ids to sets of transcripts. The format looks like
~~~
0	0
1	1
2	2
3	3
4	4
...

884398	26558,53383,53384,69915,69931,85319,109252,125730
884399	7750,35941,114698,119265
884400	9585,70083,92571,138545,138546
884401	90512,90513,134202,159456
~~~

In [ ]:
#load transcript to gene file
tr2g = {}
trlist = []
with open('./human_mouse_transcript_to_gene.tsv') as f:
    for line in f:
        l = line.split()
        tr2g[l[0]] = l[1]
        trlist.append(l[0])

genes = list(set(tr2g[t] for t in tr2g))

# load equivalence classes
ecs = {}
with open('./out_hgmm_6k/matrix.ec') as f:
    for line in f:
        l = line.split()
        ec = int(l[0])
        trs = [int(x) for x in l[1].split(',')]
        ecs[ec] = trs
        
def ec2g(ec):
    if ec in ecs:
        return list(set(tr2g[trlist[t]] for t in ecs[ec]))        
    else:
        return []

### Processing the BUS file

For these notebooks we will work with the text file that `BUStools` produces, rather than the raw `BUS` file. 
To install `BUStools` see https://github.com/BUStools/bustools

We discard any barcodes that don't have more 10 UMIs 

To produce the text file, starting with the `output.bus` file produced by kallisto, we first sort it on bustools:
```
bustools sort -o output.sorted output.bus
```
Then we convert it to txt:
```
bustools text -o output.sorted.txt output.sorted
```



In [ ]:
#sort bus file
!bustools sort -o ./out_hgmm_6k/output_sorted.bus ./out_hgmm_6k/output.bus

In [ ]:
# convert the sorted busfile to txt
!bustools text -o ./out_hgmm_6k/output_sorted.txt ./out_hgmm_6k/output_sorted.bus

# Loading the generated data  

In [ ]:
#load human_mouse transcripts

tr2g = {}
trlist = []
with open('./human_mouse_transcript_to_gene.tsv') as f:
    for line in f:
        l = line.split()
        tr2g[l[0]] = l[1]
        trlist.append(l[0])

genes = list(set(tr2g[t] for t in tr2g))

# load equivalence classes
ecs = {}
with open('./out_hgmm_6k/matrix.ec') as f:
    for line in f:
        l = line.split()
        ec = int(l[0])
        trs = [int(x) for x in l[1].split(',')]
        ecs[ec] = trs
        
def ec2g(ec):
    if ec in ecs:
        return list(set(tr2g[trlist[t]] for t in ecs[ec]))        
    else:
        return []

In [ ]:
# load kallisto bus output dataset

cell_gene = collections.defaultdict(lambda: collections.defaultdict(float))
pbar=None
pumi=None
with open('./out_hgmm_6k/output_sorted.txt') as f:
    gs = set()
    for line in f:
        l = line.split()
        barcode,umi,ec,count = line.split()
        ec = int(ec)
        
        if barcode == pbar:
            # same barcode
            if umi == pumi:
                # same UMI, let's update with intersection of genelist
                gl = ec2g(ec)
                gs.intersection_update(gl)
            else:
                # new UMI, process the previous gene set
                for g in gs:
                    cell_gene[barcode][g] += 1.0/len(gs)
                # record new umi, reset gene set
                pumi = umi
                gs = set(ec2g(ec))
        else:
            # work with previous gene list
            for g in gs:
                cell_gene[pbar][g] += 1.0/len(gs)
            
            if sum(cell_gene[pbar][g] for g in cell_gene[pbar]) < 10:
                del cell_gene[pbar]
            
            pbar = barcode
            pumi = umi
            
            gs = set(ec2g(ec))
    #remember the last gene
    for g in gs:
        cell_gene[pbar][g] += 1.0/len(gs)
        
    if sum(cell_gene[pbar][g] for g in cell_gene[pbar]) < 10:
        del cell_gene[pbar]



In [ ]:
barcode_hist = collections.defaultdict(int)
for barcode in cell_gene:
    cg = cell_gene[barcode]
    s = len([cg[g] for g in cg])
    barcode_hist[barcode] += s
    

# Take a look at the detected barcodes and genes

In [ ]:
threshold = 0 # this filters the data by gene count
bcv = [x for b,x in barcode_hist.items() if x > 0] 
_ = plt.hist(bcv,bins=100, log=True)
plt.rcParams["figure.figsize"] = [9,6]
plt.xlabel("Number of gene counts")
plt.ylabel("Number of barcodes")
plt.grid(True)
print(len(bcv))

# Export count data as `.mtx`

In [ ]:
outfile = './out_hgmm_6k/matrix.mtx'

gene_to_id = dict((g,i+1) for i,g in enumerate(genes))
barcodes_to_use = [b for b,x in barcode_hist.items() if x > 500  and x < 10000]

num_entries = 0
for barcode in barcodes_to_use:
    num_entries += len([x for x in cell_gene[barcode].values() if round(x)>0])



In [ ]:
with open(outfile, 'w') as of:
    of.write('%%MatrixMarket matrix coordinate real general\n%\n')
    #number of genes
    of.write("%d %d %d\n"%(len(genes), len(barcodes_to_use), num_entries))
    bcid = 0
    for barcode in barcodes_to_use:
        bcid += 1
        cg = cell_gene[barcode]
        gl = [(gene_to_id[g],round(cg[g])) for g in cg if round(cg[g]) > 0]
        gl.sort()
        for x in gl:
            of.write("%d %d %d\n"%(x[0],bcid,x[1]))
    

In [ ]:
gene_names = {}
with open('./human_mouse_transcript_to_gene.tsv') as f:
    f.readline()
    for line in f:
        g,t,gn = line.split()
        gene_names[g] = gn


In [ ]:
id_to_genes = dict((i,g) for (g,i) in gene_to_id.items())
gl = []
for i in range(1,len(genes)+1):
    g = id_to_genes[i]
    gid = g[:g.find('.')]
    if gid in gene_names:
        gn = gene_names[gid]
    else:
        gn = ''
    gl.append((g,gn))

with open('./out_hgmm_6k/genes.tsv','w') as of:
    for g,gn in gl:
        of.write("%s\t%s\n"%(g,gn))
        
with open('./out_hgmm_6k/barcodes.tsv','w') as of:
    of.write('\n'.join(x + '-1' for x in barcodes_to_use))
    of.write('\n')